In [ ]:
from google.colab import drive  # Se importa el drive donde estan todos los .py y las imagenes utilizadas, esta celda debe ser modificada en caso de que se quiera probar el codigo en otro equipo.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! cd /MyDrive/Duckietown_archivos && git clone https://github.com/sensxciones/Proyecto_Humedales.git

/bin/bash: line 1: cd: /MyDrive/Duckietown_archivos: No such file or directory


In [ ]:
! cd Duckietown_archivos && git pull

/bin/bash: line 1: cd: Duckietown_archivos: No such file or directory


In [1]:
# Se importar todos los modulos y funciones necesarios
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable
from torchvision import models, transforms
from PIL import Image
import sys
# sys.path.append("Duckietown_archivos")

#La siguiente importacion es de un codigo diseñado para el proyecto patimetria:
from duckies_dataset import DuckieDataset, Rescale
#Los siguientes modulos son versiones modificadas de .py de terceros adapatados para utilizar en el proyecto patimetria.
# Los modulos originales se encuentran en el siguiente git: https://github.com/adambielski/siamese-triplet
from networks import VGGEmbeddingNet, EmbeddingNet, TripletNet
from trainer import fit, train_epoch, test_epoch
from datasets import TripletMNIST
from losses import TripletLoss

In [2]:
vgg_model = VGGEmbeddingNet()

/home/felipe/Cuac/Proyecto_Humedales/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/felipe/Cuac/Proyecto_Humedales/venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/felipe/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:17<00:00, 31.9MB/s] 


In [4]:
trans = transforms.Compose([Rescale(224),
                            transforms.ToTensor()])

In [6]:
referencia_dataset = DuckieDataset('dataset-humedal/train', transform= trans)
inferencia_dataset = DuckieDataset('dataset-humedal/valid', transform= trans)

In [8]:
# Cargamos el modelo
cuda = torch.cuda.is_available()
model = TripletNet(vgg_model)
model.load_state_dict(torch.load('models/epoch-10.pt'))
model.eval()
if cuda:
    model.cuda()


In [9]:
# Define functions to extract embeddings using diferent datasets
def extract_embeddings(dataset, model, dims = 1024, opt = None):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataset), dims))
        labels = list()
        for k, it_ in enumerate(dataset):
            images = it_[0].unsqueeze_(0)
            images = images.cuda()
            target = it_[1]
            if opt is None:
                aux = model.get_embedding(images).data.cpu().numpy()
            else:
                aux = model.get_embedding(images, opt).data.cpu().numpy()
            embeddings[k] = aux.reshape(1, dims)
            labels.append(target)
    return embeddings, labels

In [24]:
embeddings, labels = extract_embeddings(inferencia_dataset, model, dims = 512)

In [26]:
import pickle
list_embeddings = []

idx_to_class = {v: k for k, v in inferencia_dataset.class_to_idx.items()}
for emb, lab in zip(embeddings, labels):
    list_embeddings.append((emb, idx_to_class[int(lab.item())]))

pickle.dump(list_embeddings, open('models/embeddings_inf.pkl', 'wb'))

In [23]:
emb_with_labels = pickle.load(open('models/embeddings.pkl', 'rb'))
emb_with_labels

[(array([0.        , 0.        , 0.        , 0.        , 0.        ,
         0.07538375, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.25784349, 0.        , 0.        , 0.        , 0.21327388,
         0.11498929, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

In [ ]:
print(labels)[0]

In [19]:
# n_closest_images: (str), (int) ---> list
# Funcion que recibe la ruta de una imagen y de manera opcional un numero entero"n" (por defecto 5), que escribe 3 listas y entrega 1:
# Near_list: lista que contiene las n distancias mas cortas con respecto a la imagen entregada
# List_labels: lista que contiene las etiquetas correspondientes a las "n" imagenes con distancias mas cortas. Esta lista es entregada por la funcion.
# Label_ubication_list: lista que entrega la posicion de las "n" imagenes mas cercanas con respecto a su etiqueta.

def n_closest_images(image, n=5):
  img = Image.open(image).convert('RGB')
  img = trans(img)
  img = img.cuda()
  emb_img = vgg_model(img.unsqueeze(0))
  embedding_img = emb_img.data.cpu().numpy()
  list_dist, list_labels, neat_list, label_ubication_list = [], [], [], []
  for i in range(embeddings.shape[0]):
    emb2 = embeddings[i,:]
    dist = np.sum((emb2 - embedding_img)**2)
    list_dist.append(dist)
  neat_list[:] = list_dist
  neat_list.sort()
  for i in range(n):
    position = list_dist.index(neat_list[i])
    label_ubication = position - labels.index(labels[position])
    label_ubication_list.append(label_ubication)
    list_labels.append(labels[position])
  #print (neat_list[:n], list_labels, label_ubication_list)
  return list_labels

In [ ]:
def closest_image_after_rotation(image):
  img = Image.open(image).convert('RGB')
  flip = transforms.RandomHorizontalFlip(p=1)
  img = flip(img)
  img = trans(img)
  img = img.cuda()
  emb_img = vgg_model(img.unsqueeze(0))
  embedding_img = emb_img.data.cpu().numpy()
  list_dist, list_labels, neat_list, label_ubication_list = [], [], [], []
  for i in range(embeddings.shape[0]):
    emb2 = embeddings[i,:]
    dist = np.sum((emb2 - embedding_img)**2)
    list_dist.append(dist)
  neat_list[:] = list_dist
  neat_list.sort()
  for i in range(1):
    position = list_dist.index(neat_list[i])
    label_ubication = position - labels.index(labels[position])
    label_ubication_list.append(label_ubication)
    list_labels.append(labels[position])
  return list_labels

In [20]:
def percentage_check_from_valid(dataset):
  Right1 = 0
  Right3 = 0
  Right5 = 0
  for i in range(len(dataset.data)):
    img = dataset.data[i]
    label= n_closest_images(img, n=10)
    if label[0] == dataset.targets[i]:
      Right1 += 1
    if dataset.targets[i] in label[:3]:
      Right3 += 1
    if dataset.targets[i] in label[:5]:
      Right5 += 1
  Total = len(dataset.targets)
  Percentage1 = 100.0*Right1/Total
  Percentage3 = 100.0*Right3/Total
  Percentage5 = 100.0*Right5/Total
  print ("% " + str(Percentage1) +" de las imagenes fueron clasificadas correctamente en top 1.")
  print ("% " + str(Percentage3) +" de las imagenes fueron clasificadas correctamente en top 3.")
  print ("% " + str(Percentage5) +" de las imagenes fueron clasificadas correctamente en top 5.")

In [21]:
percentage_check_from_valid(inferencia_dataset)

% 3.0864197530864197 de las imagenes fueron clasificadas correctamente en top 1.
% 14.814814814814815 de las imagenes fueron clasificadas correctamente en top 3.
% 19.753086419753085 de las imagenes fueron clasificadas correctamente en top 5.


In [ ]:
def percentage_check(dataset):
  Right = 0
  for i in range(len(dataset.data)):
    img = dataset.data[i]
    label= closest_image_after_rotation(img)
    if label[0] == dataset.targets[i]:
      Right += 1
  Total = len(dataset.targets)
  Percentage = 100.0*Right/Total
  print ("% " + str(Percentage) +" de las imagenes fueron clasificadas correctamente.")

In [ ]:
percentage_check(test_dataset)

In [ ]:
test_dataset.class_to_idx